In [ ]:
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import io
from docx2pdf import convert
from pathlib import Path
from nltk.corpus import stopwords
from stop_words import get_stop_words
import os
import pythoncom
import win32com.client
import base64
class ResumeAnalyzer:
    def __init__(self, resume_file):
        self.resume_file = resume_file
        
    def load_resume(self):
        # Load the resume file into memory
        with open(self.resume_file, 'r') as f:
            resume_text = f.read()
        return resume_text
    
    def process_text(self):
        pythoncom.CoInitialize()
        analyzer = self.resume_file

    # Check if the file is a PDF or DOCX file
        if analyzer.endswith('.pdf'):
            pdf_file = open(analyzer, 'rb')
        elif analyzer.endswith('.docx'):
        # Convert the DOCX file to a PDF file         
            
            
            
            word = win32com.client.Dispatch('Word.Application')
            doc = word.Documents.Add(str(analyzer))
             # Save the DOCX file as a PDF file
            pdf_file_path = str(analyzer).replace('.docx', '.pdf')
            doc.SaveAs(pdf_file_path, FileFormat=17)
        
            # Close the DOCX file
            doc.Close()
            word.Quit()
        
            # Open the PDF file
            pdf_file = open(pdf_file_path.replace('.docx', '.pdf'), 'rb')
                
            

        
          
        else:
            print("ERROR: only format of docx or PDF")
            return

    # Extract text from the PDF file
        resMgr = PDFResourceManager()
        retData = io.StringIO()
        TxtConverter = TextConverter(resMgr, retData, laparams=LAParams())
        interpreter = PDFPageInterpreter(resMgr, TxtConverter)

        for page in PDFPage.get_pages(pdf_file):
            interpreter.process_page(page)

        txt = retData.getvalue()

    # Save the extracted text to a TXT file
        txt_file = open(analyzer.replace('.pdf', '.txt'), 'w')
        txt_file.write(txt)

        return txt

    
    
    def extract_words(self, resume_text):
        # Use regular expressions to extract only words from the resume text
        words = re.findall(r'\b[a-zA-Z]+\b', resume_text)
        return words
    
    def count_data_science_words(self, words):
        # Use a set to store the words related to data science
        data_science_words = {'Python', 'R', 'SQL', 'Machine Learning',"AI","Data Science", 'Data Mining', 'Data Analysis',"DATA","spark","MYSQL","ml","algorithm","neural network","deep learning"}
        data_science_words ={data_buzz.lower() for data_buzz in data_science_words}
        # Count the number of words in the resume that are related to data science
        count = 0
        for word in words:
            if word in data_science_words:
                count += 1
        return count
    
    def process_word(self, words):        
        words = [word.lower() for word in words]
        words = [word for word in words if word not in stopwords.words('english')]
        words = [word for word in words if word not in get_stop_words('en')]
        return words
    
    def count_data_science_words_occurrences(self, words):
    # Use a set to store the words related to data science
        data_science_words = {'Python', 'R', 'SQL', 'Machine Learning',"AI","Data Science", 'Data Mining', 'Data Analysis',"DATA","spark","MYSQL","ml","algorithm","neural network","deep learning"}
        data_science_words = {word.lower() for word in data_science_words}
    
    # Create a dictionary to store the number of occurrences of each data science-related word
        word_counts = {}
        for word in words:
            if word in data_science_words:
                if word not in word_counts:
                    word_counts[word] = 1
                else:
                    word_counts[word] += 1
        word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
        return word_counts
    
    def __str__(self):
        return f'ResumeAnalyzer(file={self.resume_file})'
    
    def analyze(self):
    # Load the resume file
        resume_text = self.load_resume()
        
        # Process the text to extract the individual words
        words = self.process_text()
        
        # Count the number of data science-related words
        data_science_word_count = self.count_data_science_words(words)
        
        # Count the number of occurrences of each data science-related word
        data_science_word_occurrences = self.count_data_science_words_occurrences(words)
        
        # Return a dictionary containing the analysis results
        return {
            'data_science_word_count': data_science_word_count,
            'data_science_word_occurrences': data_science_word_occurrences,
        }

    
    
    def plot_word_cloud(self, words):
        # Generate a word cloud from the list of words
        wordcloud = WordCloud(width=800, height=400).generate(' '.join(words))
        
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()

        
def apply_word_cloud(path):
    analyzer = ResumeAnalyzer(path)
    txt = analyzer.process_text()
    try:
        words = analyzer.extract_words(txt)
        words = analyzer.process_word(words)
    # Count the number of data science-related words in the resume
        count = analyzer.count_data_science_words(words)
        word_counts = analyzer.count_data_science_words_occurrences(words)
        print(f'Number of data science-related words: {count}')
        print(f"this the word_counts {word_counts}")
        if 0.06 < count/len(words) and len(words)>50:
            name_candidate = Path(path).name.replace(".pdf","").replace(".docx","").replace(".txt","").replace(" CV","").replace(" cv","").replace("_cv","").replace("_CV","").replace("_"," ")
            print(f"This hot finding, call now to {name_candidate} !!")
        
        else:
            print("Try more buzz words!!!")
    
    # Generate and display a word cloud from the list of words in the resume
        
        analyzer.plot_word_cloud(words)
    except:
        name_candidate = Path(path).name
        print(f"ERROR: please try to cnovert file: {name_candidate} to docx or PDF only format allow")




In [ ]:
def apply_word_cloud2(path):
    analyzer = ResumeAnalyzer(path)
    txt = analyzer.process_text()
    try:
        words = analyzer.extract_words(txt)
        words = analyzer.process_word(words)
    # Count the number of data science-related words in the resume
        count = analyzer.count_data_science_words(words)
        word_counts = analyzer.count_data_science_words_occurrences(words)
        #print(f'Number of data science-related words: {count}')
        #print(f"this the word_counts {word_counts}")
        if 0.06 < count/len(words) and len(words)>50:
            name_candidate = Path(path).name.replace(".pdf","").replace(".docx","").replace(".txt","").replace(" CV","").replace(" cv","").replace("_cv","").replace("_CV","").replace("_"," ")
            #print(f"This hot finding, call now to {name_candidate} !!")
            
        else:
            name_candidate ="Try more buzz words and then your name will be apper!!!"
    
    # Generate and display a word cloud from the list of words in the resume
        
        return word_counts,name_candidate
    except:
        name_candidate = Path(path).name
        print(f"ERROR: please try to cnovert file: {name_candidate} to docx or PDF only format allow")


In [ ]:
def apply_word_cloud3(path):
    analyzer = ResumeAnalyzer(path)
    txt = analyzer.process_text()
    try:
        words = analyzer.extract_words(txt)
        words = analyzer.process_word(words)
    # Count the number of data science-related words in the resume
        count = analyzer.count_data_science_words(words)
        word_counts = analyzer.count_data_science_words_occurrences(words)
        #print(f'Number of data science-related words: {count}')
        #print(f"this the word_counts {word_counts}")
        if 0.06 < count/len(words) and len(words)>50:
            name_candidate = Path(path).name.replace(".pdf","").replace(".docx","").replace(".txt","").replace(" CV","").replace(" cv","").replace("_cv","").replace("_CV","").replace("_"," ")
            #print(f"This hot finding, call now to {name_candidate} !!")
        
        else:
            name_candidate ="Try more buzz words!!!"
            print("Try more buzz words!!!")
        if not words:
            print('Error: words list is empty or None')
            return
    # Generate and display a word cloud from the list of words in the resume
        
        #plot_word_cloud(words)
        
        wordcloud = WordCloud(width=800, height=400).generate(' '.join(words))
        
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()  

        # Save the word cloud to a file
        template_dir = os.path.dirname(path)
        image_file_path = os.path.join(template_dir, "wordcloud.png")
        # Display the word cloud
        try:
    # Save the word cloud to a file
            wordcloud.to_file(image_file_path)
        except Exception as e:
    # Print an error message if the file could not be saved
            print(f'Error: Could not save word cloud to {image_file_path}: {e}')
    except:
        name_candidate = Path(path).name
        print(f"ERROR: please try to cnovert file: {name_candidate} to docx or PDF only format allow")


In [ ]:
paths = [ r'C:\Users\יותם בראון\Desktop\Yotam Braun CV.pdf' ,r'C:\Users\יותם בראון\Desktop\Shachar_CV.pdf',r'C:\Users\יותם בראון\Desktop\simple_test.txt']

for path in paths:
    apply_word_cloud(path)

In [ ]:
paths = [ r'C:\Users\יותם בראון\Desktop\Yotam Braun CV.pdf' ,r'C:\Users\יותם בראון\Desktop\Shachar_CV.pdf',r'C:\Users\יותם בראון\Desktop\simple_test.txt']

for path in paths:
    apply_word_cloud(path)

In [ ]:
from flask import Flask, request, render_template,session

app = Flask(__name__, template_folder=r'C:\Users\יותם בראון\Desktop\templates')
IMG_FOLDER = os.path.join('static', 'IMG')
UPLOAD_FOLDER = os.path.join('staticFiles', 'uploads')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
@app.route('/')
def index():
    
    return render_template('index.html')

@app.route('/analyze', methods=["GET",'POST'])
def analyze():
    # Get the uploaded file from the request data
    file = request.files['resume_file']
    file_path1 = os.path.join(r'C:\Users\יותם בראון\Desktop\templates',file.filename)
    # Save the file to a temporary location on the server
    file_path = os.path.join(r'C:\Users\יותם בראון\Desktop\templates\tmp', file.filename)
    file.save(file_path)
    print(file_path)
    # Create a ResumeAnalyzer object and analyze the resume
    #analyzer = ResumeAnalyzer(file_path)
    #analysis = analyzer.analyze()
    analysis=apply_word_cloud2(file_path)[0]
    print(analysis)
    analysis1=apply_word_cloud2(file_path)[1]
    print(analysis1)
    analysis_2=apply_word_cloud3(file_path1)
 
    # Render the results page and pass the analysis results as data
    response = render_template('results.html', analysis=analysis,analysis1=analysis1)

    return response

@app.route('/show_image', methods=["GET", "POST"])
def displayImage():
    img_file_path = session.get("uploaded_img_file_path", None)
    # Save uploaded image file to UPLOAD_FOLDER if a file was uploaded
    if request.method == 'POST':
        uploaded_img = request.files['uploaded_img']
        uploaded_img.save(os.path.join(app.config['UPLOAD_FOLDER'], 'Wordcloud.png'))
        session['uploaded_img_file_path'] = os.path.join(app.config['UPLOAD_FOLDER'], 'Wordcloud.png')
    print(img_file_path)
    if not img_file_path:
        img_file_path = r'C:\Users\יותם בראון\Desktop\templates\Wordcloud.png'
    # Render show_image.html template and pass user_image variable
    response = render_template('show_image.html', user_image=img_file_path)
    return response



if __name__ == '__main__':
    app.run()
